In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
train_images = np.load("project3_data/augmented_data_20/images_train.npy")
train_labels = np.load("project3_data/augmented_data_20/labels_train.npy")

train_images = train_images[..., np.newaxis]
train_labels = train_labels[..., np.newaxis]

U_net = tf.keras.models.load_model('models/U_net_v02_geo_sparse')

file_path = "models/U-net_v02_geo_sparse.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

MemoryError: Unable to allocate 842. MiB for an array with shape (882900992,) and data type int8

In [ ]:
print("Original label shape: %s" % (train_labels.shape,))

## Convert the labels to categorical vectors - one hot encoded

In [ ]:
num_class = 3
train_labels = train_labels[:,:,:,0] if(len(train_labels.shape) == 4) else train_labels[:,:,0]    
new_train_labels = np.reshape(train_labels, [train_labels.shape[0], train_labels.shape[1]*train_labels.shape[2]])
#new_train_labels = to_categorical(new_train_labels, num_classes=num_class, dtype='uint8')
train_labels = new_train_labels

In [ ]:
print("Vectorized new label shape: %s" % (train_labels.shape,))

## Fit the U_net_v02_geo 

In [ ]:
history = U_net.fit(train_images, train_labels, epochs= 5, validation_split=0.2, verbose=1, callbacks=callbacks_list)

U_net.save_weights("models/U_net_v02_geo_sparse.h5")

## Get predicted labels

In [ ]:
images_test = np.load("project3_data/augmented_data_20/images_test.npy")
images_test = images_test[..., np.newaxis]

In [ ]:
U_net.load_weights("U_net_v02_geo_sparse.h5")
labels_test_pred = U_net.predict(images_test)

In [ ]:
np.save("project3_data/augmented_data_20/labels_test_pred_sparse", labels_test_pred)

## Evaluate the U-net

In [ ]:
labels_test_pred = np.load("project3_data/augmented_data_20/labels_test_pred_sparse.npy")

In [ ]:
labels_test = np.load("project3_data/augmented_data_20/labels_test.npy")
labels_test = labels_test#[..., np.newaxis]
print("Shape of test labels: %s \nShape of predicted labels: %s" % (labels_test.shape, labels_test_pred.shape))
new_labels_test = np.reshape(labels_test, [labels_test.shape[0], labels_test.shape[1]*labels_test.shape[2]])
print("New test labels shape: %s " % (new_labels_test.shape,))

In [ ]:
labels_test_pred[0][0]

In [ ]:
labels_test_pred = np.argmax(labels_test_pred, axis=-1)
print("New predicted labels shape: %s " % (labels_test_pred.shape,))

In [ ]:
def get_scores(y_true, y_pred):
    results=[]
    y_true=y_true.flatten()
    y_pred=y_pred.flatten()
    #Overall precision
    results.append(metrics.precision_score(y_true, y_pred, average='micro'))
    #Per-class precision
    results.append(metrics.precision_score(y_true, y_pred, average='macro'))
    #jaccard (right average?)
    results.append(metrics.jaccard_score(y_true, y_pred, average='macro'))
    return results
get_scores(new_labels_test, labels_test_pred)

## Plot outcome

In [ ]:
# reshape back to original dimensions
labels_test_pred_org = np.reshape(labels_test_pred, [labels_test_pred.shape[0], 256, 256])
labels_test_pred_org.shape

In [ ]:
np.unique(labels_test_pred_org)

In [ ]:
ind = 2
plt.subplot(1,2,1)
plt.imshow(labels_test[ind])
plt.subplot(1,2,2)
plt.imshow(labels_test_pred_org[ind])

In [ ]:
classes=[0,1,2]

con_mat = tf.math.confusion_matrix(labels=new_labels_test.flatten(), predictions=labels_test_pred.flatten()).numpy()

# Normalization of Confusion Matrix to the interpretation of which class is being misclassified.
con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
 
con_mat_df = pd.DataFrame(con_mat_norm, index = classes, columns = classes)

# plot confusion matrix
figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()